# Limpieza y unión de los datos de Airbnb a nivel punto. De 2019 a 2024

In [6]:
#Importar librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from pandas import read_csv


En este punto 

En este primer paso lo primero que hago es unir todos los archivos de los *listings* de Airbnb que tengo a través de los años. Asimismo, desde un inicio especifíco cuáles son las columnas/variables que necesitaré.

In [ ]:
#Definir la carpeta con las bases de datos
carpeta = r"C:\Users\santo\Documents\CIDE\Tesis\datos_airbnb\2019_2021"

In [7]:
# Lista todos los archivos en la carpeta que empiecen con 'listings'
archivos = sorted([base for base in os.listdir(carpeta) if base.startswith("listings") and base.endswith(".csv")])

# Lista vacía para ir guardando los DataFrames
dataframes = []

#Leer y guardar los archivos
for archivo in archivos:
    ruta_completa = os.path.join(carpeta, archivo) #Como los archivos no están en la misma carpeta, necesito también la ubicación de la carpeta og
    fecha_recabada = archivo.replace("listings_", "").replace(".csv", "") #Crear una nueva variable que identifica la fecha del archivo
    df = pd.read_csv(ruta_completa, usecols=['id', 'latitude', 'longitude', 'host_since', 'room_type', 'last_review'])
    df["fecha_recabada"] = fecha_recabada #Pegar la nueva columna para identificar la fecha de recabación
    dataframes.append(df)
    
#Concatenar los archivos / Unir las bases de datos en una sola
df_concatenado = pd.concat(dataframes, ignore_index = True)
df_concatenado

,id,host_since,latitude,longitude,room_type,last_review,fecha_recabada
0,14714,2009-11-26,19.430350,-99.155110,Private room,NaN,2019_03
1,22787,2010-03-03,19.440760,-99.163240,Private room,2017-02-17,2019_03
2,33681,2010-06-16,19.272150,-99.218480,Private room,NaN,2019_03
3,35797,2010-06-28,19.383990,-99.273350,Entire home/apt,NaN,2019_03
4,44616,2010-08-09,19.410060,-99.176450,Private room,2019-02-16,2019_03
...,...,...,...,...,...,...,...
515863,1318906185008171760,2020-09-26,19.436998,-99.196733,Private room,NaN,2024_12
515864,1318960023050066631,2023-07-13,19.449230,-99.136170,Entire home/apt,NaN,2024_12
515865,1318997244771691320,2019-09-10,19.316820,-99.116781,Shared room,NaN,2024_12
515866,1319078571963617776,2015-06-07,19.400096,-99.172981,Entire home/apt,NaN,2024_12


### Exportar la base de datos de Airbnb ya unida (2019-2024)


Exporto la base de datos de Airbnb de 2019-2024 para poder llamarla más fácil. 

Nota: No lo guardo como *airbnb_2019_03_2024_12* porque si vuelvo a correr el código, como una de las condiciones para juntar los archivos era tomar todos los que empiezan con *listings*, entonces voy a copiar dos veces la base. 

In [8]:
#Guardamos el csv ya completo en la carpeta con los demás listings
df_concatenado.to_csv(os.path.join(carpeta, "airbnb_2019_03_2024_12.csv"), index = False, encoding='utf-8-sig')

## Crear una nueva columna para condicionar los Airbnb's que sí están activos

Debido a que la base de *listings* parece que recaba Airbnb's que ya no se rentan, con esta parte del código quiero crear una condición para saber si un Airbnb continúa siendo rentado en la base de datos. 

Para identificar los Airbnb's activos voy a crear 4 métodos. 
1. El primero es tomar todos los *listings* que hay en la base en cada mes como la oferta de Airbnb.
2. El segundo es que un *listings/Airbnb* se considera activo si en los últimos $3$ meses ha recibido una *review* o si el *host* tiene $3$ meses como mucho de haberse unido a la plataforma.
3. El tercero es que un *listings/Airbnb* se considera activo si en los últimos $6$ meses ha recibido una *review* o si el *host* tiene $6$ meses como mucho de haberse unido a la plataforma.
4. El cuarto es que un *listings/Airbnb* se considera activo si en los últimos $12$ meses ($1$ año) ha recibido una *review* o si el *host* tiene $12$ meses ($1$ año) como mucho de haberse unido a la plataforma.


#### Cargar el archivo

In [3]:
#Cargar el CSV
archivo = "airbnb_2019_03_2024_12.csv"

ruta_completa = os.path.join(carpeta, archivo)
df_concatenado = pd.read_csv(ruta_completa)

In [9]:
df_concatenado

,id,host_since,latitude,longitude,room_type,last_review,fecha_recabada
0,14714,2009-11-26,19.430350,-99.155110,Private room,NaN,2019_03
1,22787,2010-03-03,19.440760,-99.163240,Private room,2017-02-17,2019_03
2,33681,2010-06-16,19.272150,-99.218480,Private room,NaN,2019_03
3,35797,2010-06-28,19.383990,-99.273350,Entire home/apt,NaN,2019_03
4,44616,2010-08-09,19.410060,-99.176450,Private room,2019-02-16,2019_03
...,...,...,...,...,...,...,...
515863,1318906185008171760,2020-09-26,19.436998,-99.196733,Private room,NaN,2024_12
515864,1318960023050066631,2023-07-13,19.449230,-99.136170,Entire home/apt,NaN,2024_12
515865,1318997244771691320,2019-09-10,19.316820,-99.116781,Shared room,NaN,2024_12
515866,1319078571963617776,2015-06-07,19.400096,-99.172981,Entire home/apt,NaN,2024_12


### Homologar las fechas

In [10]:
# Convertir columnas a fecha. Con 'coerce' indíco que si hay un error al transformar los datos, entonces ponga NaT en lugar de mandar error
df_concatenado['host_since'] = pd.to_datetime(df_concatenado['host_since']) 
df_concatenado['last_review'] = pd.to_datetime(df_concatenado['last_review'])

# Convertir 'fecha_recabada' a fecha con el día 1 para tener formato completo
df_concatenado['fecha_recabada_dt'] = pd.to_datetime(df_concatenado['fecha_recabada'] + "_01", format="%Y_%m_%d")

### Crear la función para utilizar los 4 métodos de identificación de Airbnb activo

In [11]:
#Cargar libreria
from functools import partial

#Definir la función
def evaluar_actividad_dinamica(row, offset):
    fecha = row['fecha_recabada_dt']
    review = row['last_review']
    host = row['host_since']
    
    if pd.notnull(review):
        return review >= (fecha - offset)
    else:
        return host >= (fecha - offset)
    
#Crear las funciones/condiciones personalizadas. partial sirve para crear una función con uno o más parametros predefinidos

evaluar_3_months = partial(evaluar_actividad_dinamica, offset = pd.DateOffset(months = 3)) 
evaluar_6_months = partial(evaluar_actividad_dinamica, offset = pd.DateOffset(months = 6))
evaluar_1_year = partial(evaluar_actividad_dinamica, offset = pd.DateOffset(years = 1))

#Crear las nuevas columnas con las condiciones de las funciones
df_concatenado['activo_3m'] = df_concatenado.apply(evaluar_3_months, axis=1)
df_concatenado['activo_6m'] = df_concatenado.apply(evaluar_6_months, axis=1)
df_concatenado['activo_1y'] = df_concatenado.apply(evaluar_1_year, axis=1)

#Crear una columna nueva para el primer método de conteo de Airbnb
df_concatenado['activo_siempre'] = True

In [13]:
df_concatenado

,id,host_since,latitude,longitude,room_type,last_review,fecha_recabada,fecha_recabada_dt,activo_3m,activo_6m,activo_1y,activo_siempre
0,14714,2009-11-26,19.430350,-99.155110,Private room,NaT,2019_03,2019-03-01,False,False,False,True
1,22787,2010-03-03,19.440760,-99.163240,Private room,2017-02-17,2019_03,2019-03-01,False,False,False,True
2,33681,2010-06-16,19.272150,-99.218480,Private room,NaT,2019_03,2019-03-01,False,False,False,True
3,35797,2010-06-28,19.383990,-99.273350,Entire home/apt,NaT,2019_03,2019-03-01,False,False,False,True
4,44616,2010-08-09,19.410060,-99.176450,Private room,2019-02-16,2019_03,2019-03-01,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
515863,1318906185008171760,2020-09-26,19.436998,-99.196733,Private room,NaT,2024_12,2024-12-01,False,False,False,True
515864,1318960023050066631,2023-07-13,19.449230,-99.136170,Entire home/apt,NaT,2024_12,2024-12-01,False,False,False,True
515865,1318997244771691320,2019-09-10,19.316820,-99.116781,Shared room,NaT,2024_12,2024-12-01,False,False,False,True
515866,1319078571963617776,2015-06-07,19.400096,-99.172981,Entire home/apt,NaT,2024_12,2024-12-01,False,False,False,True


## Preparar la base final a exportar

En esta parte quito las columnas que no ocuparé para la suma final   

In [14]:
#Quitar columnas
columnas_a_eliminar = ['host_since', 'last_review', 'fecha_recabada']
df_limpio = df_concatenado.drop(columns = columnas_a_eliminar)

In [15]:
df_limpio

,id,latitude,longitude,room_type,fecha_recabada_dt,activo_3m,activo_6m,activo_1y,activo_siempre
0,14714,19.430350,-99.155110,Private room,2019-03-01,False,False,False,True
1,22787,19.440760,-99.163240,Private room,2019-03-01,False,False,False,True
2,33681,19.272150,-99.218480,Private room,2019-03-01,False,False,False,True
3,35797,19.383990,-99.273350,Entire home/apt,2019-03-01,False,False,False,True
4,44616,19.410060,-99.176450,Private room,2019-03-01,True,True,True,True
...,...,...,...,...,...,...,...,...,...
515863,1318906185008171760,19.436998,-99.196733,Private room,2024-12-01,False,False,False,True
515864,1318960023050066631,19.449230,-99.136170,Entire home/apt,2024-12-01,False,False,False,True
515865,1318997244771691320,19.316820,-99.116781,Shared room,2024-12-01,False,False,False,True
515866,1319078571963617776,19.400096,-99.172981,Entire home/apt,2024-12-01,False,False,False,True


# Exportar la base de Airbnb final

In [17]:
carpeta_final = r"C:\Users\santo\Documents\CIDE\Tesis\datos_unidos"

df_limpio.to_csv(os.path.join(carpeta_final, "airbnb_limpio.csv"), index = False, encoding='utf-8-sig')